In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_path = "/content/Major_Safety_Events.csv"
df = pd.read_csv(data_path)

df.head()

<ipython-input-2-48ae5cc9c05d>:2: DtypeWarning: Columns (18,20,24,34,38,45,46,49,53,54,60,63,64,69,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


,NTD ID,Agency,Primary UZA UACE Code,Rail/Bus/Ferry,Mode Name,Mode,TOS,Fixed Route Flag,Year,Event Description,...,Occupant of Other Vehicle Injuries,Occupant of Other Vehicle Serious Injuries,Other Injuries,Other Serious Injuries,Suicide Injuries,Suicide Serious Injuries,Trespasser Inuries (Subtotal),Trespasser Serious Injuries (Subtotal),Total Serious Injuries,Person List
0,20080,New Jersey Transit Corporation,63217,Bus,Bus,MB,DO,True,2024,Bus was parked in <REDACTED> <REDACTED> and an...,...,0,0,0,0,0,0,0,0,0,NaN
1,40035,Central Florida Regional Transportation Authority,65863,Bus,Demand Response,DR,DO,False,2024,Another vehicle made other front contact with ...,...,1,0,0,0,0,0,0,0,0,I Operator Inside Adult (19 to 60) F ; I ...
2,50027,Metro Transit,57628,Rail,Light Rail,LR,DO,True,2024,Officers responded to a shooting . Suspect was...,...,0,0,0,0,0,0,0,0,0,F Passenger Adult (19 to 60) M
3,90026,San Diego Metropolitan Transit System,78661,Rail,Light Rail,LR,DO,True,2024,San Diego Police took a female into custody af...,...,0,0,0,0,0,0,0,0,0,I Wait Leave Adult (19 to 60) M Y
4,20080,New Jersey Transit Corporation,63217,Bus,Bus,MB,DO,True,2024,operator was driving down <REDACTED> <REDACTED...,...,1,0,0,0,0,0,0,0,0,I Occ Other Veh Adult (19 to 60) M


In [3]:
column_names = df.columns.tolist()

column_names

['NTD ID',
 'Agency',
 'Primary UZA UACE Code',
 'Rail/Bus/Ferry',
 'Mode Name',
 'Mode',
 'TOS',
 'Fixed Route Flag',
 'Year',
 'Event Description',
 'Incident Number',
 'Event Date',
 'Event Time',
 'Event Type',
 'Event Type Group',
 'Transit Worker Type',
 'Transit Worker Assault Detail Type',
 'Transit Worker Assault Flag',
 'Other Event Type Description',
 'Event Category',
 'Runaway Train Flag',
 'Safety/Security',
 'Collision With',
 'Property Damage',
 'Substantial Property Damage Type',
 'Total Injuries',
 'Total Fatalities',
 'Towed (Y/N)',
 'Number of Transit Vehicles Involved',
 'Number of Non-Transit Vehicles Involved',
 'Number of Cars on Involved Transit Vehicles',
 'Number of Derailed Cars on Involved Transit Vehicles',
 'Non-Transit Vehicle Type List',
 'Location Type',
 'Multiple Locations List',
 'Approximate Address',
 'Latitude',
 'Longitude',
 'LatLon',
 'Evacuation',
 'Evacuation for Life Safety (Y/N)',
 'Evacuation to Right-of-Way',
 'Evacuation Comment',
 'Sel

In [4]:
columns_to_keep = [
    'NTD ID',
    'Primary UZA UACE Code',
    'Rail/Bus/Ferry',
    'Mode Name',
    'Mode',
    'TOS',
    'Year',
    'Event Date',
    'Event Time',
    'Event Type',
    'Event Type Group',
    'Safety/Security',
    'Property Damage',
    'Total Injuries',
    'Total Fatalities',
    'Towed (Y/N)',
    'Number of Transit Vehicles Involved',
    'Number of Non-Transit Vehicles Involved',
    'Number of Cars on Involved Transit Vehicles',
    'Non-Transit Vehicle Type List',
    'Location Type',
    'Latitude',
    'Longitude',
    'Weather',
    'Lighting',
    'Road Configuration',
    'Path Condition',
    'Right of Way Condition',
    'Intersection Control Device',
    'Transit Vehicle Action',
    'Other Transit Vehicle Action Description',
    'Non-Transit Vehicle Action List',
    'Transit (Y/N)',
    'Fuel Type',
    'Vehicle Speed',
    'Transit Vehicle Type',
    'Non-Transit Vehicle Type',
    'Transit Vehicle Manufacturer',
    'Total Serious Injuries'
]

df = df[columns_to_keep]


In [5]:
missing_values = df.isnull().sum()
columns_to_drop = missing_values[missing_values > 0.1 * df.shape[0]].index
df = df.drop(columns=columns_to_drop)

df = df.dropna()

df = df.drop_duplicates()

# TO save the cleaned data, save to data folder as data_cleaned.csv
# df.to_csv('../data/cleaned_data.csv', index=False)

print(df.shape)

print(df.isnull().sum())

print(df.duplicated().sum())

print(df.head())

(98611, 20)
NTD ID                                         0
Primary UZA UACE Code                          0
Rail/Bus/Ferry                                 0
Mode Name                                      0
Mode                                           0
TOS                                            0
Year                                           0
Event Date                                     0
Event Time                                     0
Event Type                                     0
Event Type Group                               0
Safety/Security                                0
Total Injuries                                 0
Total Fatalities                               0
Towed (Y/N)                                    0
Number of Transit Vehicles Involved            0
Number of Non-Transit Vehicles Involved        0
Number of Cars on Involved Transit Vehicles    0
Location Type                                  0
Total Serious Injuries                         0
dtype: i

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [7]:
DAMAGE_THRESHOLD = 25_000

df["Major_Incident"] = df.apply(
    lambda row: 1 if (
        row.get("Total Serious Injuries", 0) > 0
        or row.get("Total Fatalities", 0) > 0
        or row.get("Property Damage", 0) > DAMAGE_THRESHOLD
    ) else 0,
    axis=1
)


In [8]:
'''
['NTD ID', 'Primary UZA UACE Code', 'Rail/Bus/Ferry', 'Mode Name', 'Mode', 'TOS', 'Year', 'Event Date',
'Event Time', 'Event Type', 'Event Type Group', 'Safety/Security', 'Total Injuries', 'Total Fatalities',
'Towed (Y/N)', 'Number of Transit Vehicles Involved', 'Number of Non-Transit Vehicles Involved', 'Number of Cars on Involved Transit Vehicles',
'Location Type', 'Total Serious Injuries', 'Major_Incident']
'''


"\n['NTD ID', 'Primary UZA UACE Code', 'Rail/Bus/Ferry', 'Mode Name', 'Mode', 'TOS', 'Year', 'Event Date',\n'Event Time', 'Event Type', 'Event Type Group', 'Safety/Security', 'Total Injuries', 'Total Fatalities',\n'Towed (Y/N)', 'Number of Transit Vehicles Involved', 'Number of Non-Transit Vehicles Involved', 'Number of Cars on Involved Transit Vehicles',\n'Location Type', 'Total Serious Injuries', 'Major_Incident']\n"

In [9]:
y = df["Major_Incident"]
X = df.drop(["Major_Incident"], axis=1)

df.drop(["Event Date"], axis=1)

known_categorical = [
    'NTD ID',
    'Primary UZA UACE Code',
    'Rail/Bus/Ferry',
    'Mode Name',
    'Mode',
    'TOS',
    'Event Type',
    'Event Type Group',
    'Safety/Security',
    'Towed (Y/N)',
    'Location Type'
]

all_columns = X.columns.tolist()
numeric_cols = [col for col in all_columns if col not in known_categorical]

X[numeric_cols] = X[numeric_cols].fillna(0)


for col in known_categorical:
    # Replace NaN with a placeholder category
    X[col] = X[col].fillna("MissingCategory")
    # Factorize (maps each category to an integer code)
    X[col] = X[col].astype(str).factorize()[0]

# factorize event date and time
X["Event Date"] = X["Event Date"].astype(str).fillna("MissingDate").factorize()[0]
X["Event Time"] = X["Event Time"].astype(str).fillna("MissingTime").factorize()[0]

In [10]:
known_categorical = [col for col in known_categorical if col in X.columns]

all_columns = X.columns.tolist()
numeric_cols = [col for col in all_columns if col not in known_categorical]

# Fill numeric NaNs with 0 (simple strategy)
X[numeric_cols] = X[numeric_cols].fillna(0)

# Factorize categorical columns
for col in known_categorical:
    X[col] = X[col].fillna("MissingCategory")
    # factorize() -> returns (array_of_codes, array_of_unique_vals)
    X[col] = X[col].astype(str).factorize()[0]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # keep class distribution consistent
)

logreg = LogisticRegression(max_iter=500)
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9777924250874613

Confusion Matrix:
 [[18648    40]
 [  398   637]]

Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99     18688
           1       0.94      0.62      0.74      1035

    accuracy                           0.98     19723
   macro avg       0.96      0.81      0.87     19723
weighted avg       0.98      0.98      0.98     19723



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
